In [1]:
import pandas as pd
import sqlite3

orders = pd.read_csv("orders.csv")
users = pd.read_json("users.json")



In [2]:
import sqlite3
import pandas as pd

# Connect to SQLite (creates DB if not exists)
conn = sqlite3.connect("restaurants.db")
cursor = conn.cursor()

# Read SQL file
with open("restaurants.sql", "r", encoding="utf-8") as f:
    sql_script = f.read()

# Execute SQL script (creates table + inserts data)
cursor.executescript(sql_script)

conn.commit()


OperationalError: table restaurants already exists

In [3]:
pd.read_sql(
    "SELECT name FROM sqlite_master WHERE type='table';",
    conn
)


,name
0,restaurants


In [4]:
restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
restaurants.head()


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [6]:

# Merge datasets
final_df = (
    orders
    .merge(users, on="user_id", how="left")
    .merge(restaurants, on="restaurant_id", how="left")
)

# Save final dataset
final_df.to_csv("final_food_delivery_dataset.csv", index=False)


In [7]:
final_df.info()
final_df.isna().sum()
final_df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   order_id           10000 non-null  int64  
 1   user_id            10000 non-null  int64  
 2   restaurant_id      10000 non-null  int64  
 3   order_date         10000 non-null  object 
 4   total_amount       10000 non-null  float64
 5   restaurant_name_x  10000 non-null  object 
 6   name               10000 non-null  object 
 7   city               10000 non-null  object 
 8   membership         10000 non-null  object 
 9   restaurant_name_y  10000 non-null  object 
 10  cuisine            10000 non-null  object 
 11  rating             10000 non-null  float64
dtypes: float64(2), int64(3), object(7)
memory usage: 937.6+ KB


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [9]:
final_df[final_df["membership"]=="Gold"] \
.groupby("city")["total_amount"] \
.sum().sort_values(ascending=False)


city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64

In [10]:
final_df.groupby("cuisine")["total_amount"] \
.mean().sort_values(ascending=False)


cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64

In [11]:
user_spend = final_df.groupby("user_id")["total_amount"].sum()
user_spend[user_spend > 1000].nunique()


2543

In [20]:
final_df["rating_bucket"] = pd.cut(
    final_df["rating"],
    bins=[3.0, 3.5,3.6, 4.0,4.1, 4.5,4.6, 5.0]
)

final_df.groupby("rating_bucket")["total_amount"] \
.sum().sort_values(ascending=False)


C:\Users\pavanmajji\AppData\Local\Temp\ipykernel_31444\313397474.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  final_df.groupby("rating_bucket")["total_amount"] \


rating_bucket
(3.0, 3.5]    1881754.57
(4.6, 5.0]    1701162.78
(4.1, 4.5]    1579475.41
(3.6, 4.0]    1453300.47
(4.5, 4.6]     495867.97
(4.0, 4.1]     380850.85
(3.5, 3.6]     264193.94
Name: total_amount, dtype: float64

In [13]:
final_df[final_df["membership"]=="Gold"] \
.groupby("city")["total_amount"] \
.mean().sort_values(ascending=False)


city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64

In [14]:
pd.concat([
    final_df.groupby("cuisine")["restaurant_id"].nunique(),
    final_df.groupby("cuisine")["total_amount"].sum()
], axis=1).rename(
    columns={"restaurant_id":"restaurant_count",
             "total_amount":"revenue"}
).sort_values("restaurant_count")


,restaurant_count,revenue
cuisine,,
Chinese,120,1930504.65
Indian,126,1971412.58
Italian,126,2024203.80
Mexican,128,2085503.09


In [15]:
round(
    (final_df[final_df["membership"]=="Gold"].shape[0] / final_df.shape[0]) * 100
)


50

In [17]:
restaurant_stats = final_df.groupby("restaurant_name_x").agg(
    avg_order=("total_amount", "mean"),
    orders=("order_id", "count")
)

restaurant_stats[restaurant_stats["orders"] < 20] \
.sort_values("avg_order", ascending=False)


,avg_order,orders
restaurant_name_x,,
Hotel Dhaba Multicuisine,1040.222308,13
Sri Mess Punjabi,1029.180833,12
Ruchi Biryani Punjabi,1002.140625,16
Sri Delights Pure Veg,989.467222,18
Classic Kitchen Family Restaurant,973.167895,19
...,...,...
Annapurna Tiffins Punjabi,621.828947,19
Darbar Tiffins Non-Veg,596.815556,18
Darbar Restaurant Punjabi,589.972857,14


In [21]:
options = [
    "Grand Cafe Punjabi",
    "Grand Restaurant South Indian",
    "Ruchi Mess Multicuisine",
    "Ruchi Foods Chinese"
]

restaurant_stats.loc[options] \
.sort_values("avg_order", ascending=False)


,avg_order,orders
restaurant_name_x,,
Ruchi Mess Multicuisine,851.226250,40
Grand Restaurant South Indian,842.567586,29
Grand Cafe Punjabi,765.409063,32
Ruchi Foods Chinese,686.603158,19


In [18]:
final_df.groupby(["membership", "cuisine"])["total_amount"] \
.sum().sort_values(ascending=False)


membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [19]:
final_df["order_date"] = pd.to_datetime(final_df["order_date"])
final_df["quarter"] = final_df["order_date"].dt.to_period("Q")

final_df.groupby("quarter")["total_amount"] \
.sum().sort_values(ascending=False)


C:\Users\pavanmajji\AppData\Local\Temp\ipykernel_31444\1682843972.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  final_df["order_date"] = pd.to_datetime(final_df["order_date"])


quarter
2023Q3    2037385.10
2023Q4    2018263.66
2023Q1    1993425.14
2023Q2    1945348.72
2024Q1      17201.50
Freq: Q-DEC, Name: total_amount, dtype: float64

In [23]:
final_df[final_df["membership"] == "Gold"].shape[0]


4987

In [24]:
round(
    final_df[final_df["city"] == "Hyderabad"]["total_amount"].sum()
)


1889367

In [25]:
final_df["user_id"].nunique()


2883

In [26]:
round(
    final_df[final_df["membership"] == "Gold"]["total_amount"].mean(),
    2
)


np.float64(797.15)

In [27]:
final_df[final_df["rating"] >= 4.5].shape[0]


3374

In [28]:

top_city = (
    final_df[final_df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .idxmax()
)


In [29]:

top_city
final_df[
    (final_df["membership"] == "Gold") &
    (final_df["city"] == top_city)
].shape[0]


1337

In [30]:
final_df.shape[0]


10000